# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.16.2'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
# Do not apply data augmentation to the test set (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
# Adding the convolutional layer
# input_shape=[64, 64, 3] -> 64x64 pixels, 3 channels (RGB)
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

/Users/danielmartinezvillegas/anaconda3/envs/tf-macos/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-25 11:43:26.699454: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-10-25 11:43:26.699478: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-10-25 11:43:26.699486: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-10-25 11:43:26.699740: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-25 11:43:26.699756: I tensorflow/core/common_runtime/pluggable_dev

### Step 2 - Pooling

In [7]:
# Max pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
# flattening
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
# full connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
# output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
# compile the CNN
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
# train the CNN
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25


2024-10-25 11:43:27.508601: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/danielmartinezvillegas/anaconda3/envs/tf-macos/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.5426 - loss: 0.6913 - val_accuracy: 0.6940 - val_loss: 0.5929
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.6699 - loss: 0.6050 - val_accuracy: 0.7305 - val_loss: 0.5444
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.6982 - loss: 0.5706 - val_accuracy: 0.7375 - val_loss: 0.5558
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.7078 - loss: 0.5663 - val_accuracy: 0.7645 - val_loss: 0.5122
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.7481 - loss: 0.5095 - val_accuracy: 0.7080 - val_loss: 0.5781
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7617 - loss: 0.5024 - val_accuracy: 0.7760 - val_loss: 0.4819
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7676 - loss: 0.4761 - val_accuracy: 0.7645 - val_loss: 0.5006
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.7838 - loss: 0.4634 - val_accuracy: 

## Part 4 - Making a single prediction

In [14]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size=(64, 64))
# convert into a array
test_image = image.img_to_array(test_image)
# add batch dimension
test_image = np.expand_dims(test_image, axis=0)
# result
result = cnn.predict(test_image)
# encode the result
training_set.class_indices
if result[0][0] == 1:
	prediction = 'dog'
else:
	prediction = 'cat'


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


In [15]:
print(prediction)

dog
